In [ ]:
import lightgbm as lgb
import xgboost as xgb
import torch

def check_gpu_availability():
    try:
        # LightGBM GPU Check
        lgbm_params = {"device": "gpu"}
        lgb.LGBMClassifier(**lgbm_params)
        lgbm_gpu = True
    except Exception:
        lgbm_gpu = False

    # XGBoost GPU Check
    xgb_gpu = torch.cuda.is_available()  # XGBoost can use CUDA if available

    # PyTorch GPU Check
    torch_gpu = torch.cuda.is_available()

    gpu_status = {
        "LightGBM": lgbm_gpu,
        "XGBoost": xgb_gpu,
        "PyTorch": torch_gpu
    }

    for lib, available in gpu_status.items():
        print(f" {lib} GPU Support: {'Available' if available else 'Not Available'}")

    return gpu_status

# Run the check before training
gpu_status = check_gpu_availability()

# Example usage:
if gpu_status["LightGBM"]:
    print("Using GPU for LightGBM training...")
else:
    print("Falling back to CPU for LightGBM.")

if gpu_status["XGBoost"]:
    print("Using GPU for XGBoost training...")
else:
    print("Falling back to CPU for XGBoost.")


In [ ]:
# All columns
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#  Check GPU Availability
gpu_available = xgb.XGBClassifier().get_params().get('device', 'cpu') == 'cuda'
print(f" GPU Available: {gpu_available}")

#  Load dataset
print("\n Loading dataset...")
df = pd.read_csv("df.csv")
print(f" Dataset Loaded! Shape: {df.shape}")

#  Encode categorical labels
print("\n Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print(" Encoding complete!")

#  Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values
n_classes = len(np.unique(y))  # Number of unique classes

#  Compute Class Weights for Imbalance Handling
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in np.unique(y)}

#  Split dataset
print("\n Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f" Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

#  Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Define XGBoost Model (Handles Class Weights)
xgb_model = xgb.XGBClassifier(
    n_estimators=150, learning_rate=0.03, max_depth=7,
    tree_method="gpu_hist" if gpu_available else "hist",
    objective="multi:softprob", num_class=n_classes,
    device="cuda" if gpu_available else "cpu", n_jobs=-1
)

#  Train XGBoost with Sample Weights
print("\n Training XGBoost (Handling Class Imbalance)...")
xgb_model.fit(X_train, y_train, sample_weight=np.array([class_weight_dict[i] for i in y_train]))
print(" XGBoost Training Completed!")

#  Define Base Models (RF + XGB + Logistic Regression)
base_models = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=25, class_weight="balanced", random_state=42, n_jobs=-1)),
    ('xgb', xgb_model),
    ('logreg', LogisticRegression(class_weight="balanced", max_iter=500, solver="saga", multi_class="multinomial", n_jobs=-1))
]

#  Define Meta-Model (LightGBM)
meta_model = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.03, max_depth=7, 
                                num_leaves=40, random_state=42, 
                                is_unbalance=True,  # Helps with class imbalance
                                min_gain_to_split=0.01,  # Prevents over-pruning
                                device="gpu" if gpu_available else "cpu")

#  Define Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

#  Train Stacking Model
print("\n Training Optimized Stacking Classifier...")
start_time = time.time()
stacking_model.fit(X_train, y_train)
end_time = time.time()
print(f" Optimized Stacking Classifier Training Completed in {end_time - start_time:.2f} seconds!")

#  Predictions
print("\n Making predictions...")
y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)

#  Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 {model_name} Results:")
    print(results)
    print(f"\n Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

#  Evaluate Stacked Model
print("\ Evaluating Optimized Stacking Classifier...")
evaluate_model(y_test, y_pred, y_pred_proba, "Stacking Classifier")

print("\n Training & Evaluation Complete! ")


In [ ]:
# Gass columns dropped
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#  Check GPU Availability
gpu_available = xgb.XGBClassifier().get_params().get('device', 'cpu') == 'cuda'
print(f" GPU Available: {gpu_available}")

#  Load dataset
print("\n Loading dataset...")
df = pd.read_csv("df.csv")
print(f" Dataset Loaded! Shape: {df.shape}")

#  Drop columns related to gas usage
gas_columns = [
    "Gas:Facility [kW](Hourly)",
    "Heating:Gas [kW](Hourly)",
    "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]
df = df.drop(columns=gas_columns, errors='ignore')

#  Encode categorical labels
print("\n Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print(" Encoding complete!")

#  Separate features and target
X = df.drop(columns=["theft"])
y = df["theft"]
n_classes = len(np.unique(y))  # Number of unique classes

#  Compute Class Weights for Imbalance Handling
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in np.unique(y)}

#  Split dataset
print("\n Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f" Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

#  Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Define XGBoost Model (Handles Class Weights)
xgb_model = xgb.XGBClassifier(
    n_estimators=150, learning_rate=0.03, max_depth=7,
    tree_method="gpu_hist" if gpu_available else "hist",
    objective="multi:softprob", num_class=n_classes,
    device="cuda" if gpu_available else "cpu", n_jobs=-1
)

#  Train XGBoost with Sample Weights
print("\n Training XGBoost (Handling Class Imbalance)...")
xgb_model.fit(X_train, y_train, sample_weight=np.array([class_weight_dict[i] for i in y_train]))
print(" XGBoost Training Completed!")

#  Define Base Models (RF + XGB + Logistic Regression)
base_models = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=25, class_weight="balanced", random_state=42, n_jobs=-1)),
    ('xgb', xgb_model),
    ('logreg', LogisticRegression(class_weight="balanced", max_iter=500, solver="saga", multi_class="multinomial", n_jobs=-1))
]

#  Define Meta-Model (LightGBM)
meta_model = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.03, max_depth=7, 
                                num_leaves=40, random_state=42, 
                                is_unbalance=True,  # Helps with class imbalance
                                min_gain_to_split=0.01,  # Prevents over-pruning
                                device="gpu" if gpu_available else "cpu")

#  Define Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

#  Train Stacking Model
print("\n Training Optimized Stacking Classifier...")
start_time = time.time()
stacking_model.fit(X_train, y_train)
end_time = time.time()
print(f" Optimized Stacking Classifier Training Completed in {end_time - start_time:.2f} seconds!")

#  Predictions
print("\n Making predictions...")
y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)

#  Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n {model_name} Results:")
    print(results)
    print(f"\n Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

#  Evaluate Stacked Model
print("\n Evaluating Optimized Stacking Classifier...")
evaluate_model(y_test, y_pred, y_pred_proba, "Stacking Classifier")

print("\n Training & Evaluation Complete! ")


In [ ]:
# From Optuna Params all columns
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#  Check GPU Availability
gpu_available = xgb.XGBClassifier().get_params().get('device', 'cpu') == 'cuda'
print(f" GPU Available: {gpu_available}")

#  Load dataset
print("\n Loading dataset...")
df = pd.read_csv("df.csv")
print(f" Dataset Loaded! Shape: {df.shape}")

#  Encode categorical labels
print("\n Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print(" Encoding complete!")

#  Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values
n_classes = len(np.unique(y))  # Number of unique classes

#  Compute Class Weights for Imbalance Handling
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in np.unique(y)}

#  Split dataset
print("\n Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f" Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

#  Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Define XGBoost Model (Updated Parameters)
xgb_model = xgb.XGBClassifier(
    learning_rate=0.0997, max_depth=11, min_child_weight=4, subsample=0.8696,
    colsample_bytree=0.6797, n_estimators=126, 
    tree_method="gpu_hist" if gpu_available else "hist",
    objective="multi:softprob", num_class=n_classes,
    device="cuda" if gpu_available else "cpu", n_jobs=-1
)

#  Train XGBoost with Sample Weights
print("\n Training XGBoost (Handling Class Imbalance)...")
xgb_model.fit(X_train, y_train, sample_weight=np.array([class_weight_dict[i] for i in y_train]))
print(" XGBoost Training Completed!")

#  Define Base Models (RF + XGB + Logistic Regression)
base_models = [
    ('rf', RandomForestClassifier(
        n_estimators=184, max_depth=11, min_samples_split=18, min_samples_leaf=4, max_features='sqrt',
        class_weight="balanced", random_state=42, n_jobs=-1
    )),
    ('xgb', xgb_model),
    ('logreg', LogisticRegression(
        solver="lbfgs", max_iter=1000, C=13812.3,
        class_weight="balanced", multi_class="multinomial", n_jobs=-1
    ))
]

#  Define Meta-Model (Updated LightGBM Parameters)
meta_model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.0987, num_leaves=190, max_depth=11, min_data_in_leaf=12,
    max_bin=356, feature_fraction=0.546, random_state=42,
    is_unbalance=True,  # Helps with class imbalance
    min_gain_to_split=0.01,  # Prevents over-pruning
    device="gpu" if gpu_available else "cpu"
)

#  Define Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

#  Train Stacking Model
print("\n Training Optimized Stacking Classifier...")
start_time = time.time()
stacking_model.fit(X_train, y_train)
end_time = time.time()
print(f" Optimized Stacking Classifier Training Completed in {end_time - start_time:.2f} seconds!")

#  Predictions
print("\n Making predictions...")
y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)

#  Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 {model_name} Results:")
    print(results)
    print(f"\n Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

#  Evaluate Stacked Model
print("\n Evaluating Optimized Stacking Classifier...")
evaluate_model(y_test, y_pred, y_pred_proba, "Stacking Classifier")

print("\n Training & Evaluation Complete! ")


In [ ]:
#From Optuna Params without gas columns
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#  Check GPU Availability
gpu_available = xgb.XGBClassifier().get_params().get('device', 'cpu') == 'cuda'
print(f" GPU Available: {gpu_available}")

#  Load dataset
print("\n Loading dataset...")
df = pd.read_csv("df.csv")
print(f" Dataset Loaded! Shape: {df.shape}")

#  Drop columns related to gas usage
gas_columns = [
    "Gas:Facility [kW](Hourly)",
    "Heating:Gas [kW](Hourly)",
    "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]
df = df.drop(columns=gas_columns, errors='ignore')

#  Encode categorical labels
print("\n Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print(" Encoding complete!")

#  Separate features and target
X = df.drop(columns=["theft"])
y = df["theft"]
n_classes = len(np.unique(y))  # Number of unique classes

#  Compute Class Weights for Imbalance Handling
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in np.unique(y)}

#  Split dataset
print("\n Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f" Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

#  Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Define XGBoost Model (Handles Class Weights)
xgb_model = xgb.XGBClassifier(
    learning_rate=0.09144751092423227,
    max_depth=12,
    min_child_weight=7,
    subsample=0.8781111961629534,
    colsample_bytree=0.8584849992349836,
    n_estimators=200,
    tree_method="gpu_hist" if gpu_available else "hist",
    objective="multi:softprob", num_class=n_classes,
    device="cuda" if gpu_available else "cpu", n_jobs=-1
)

#  Train XGBoost with Sample Weights
print("\n Training XGBoost (Handling Class Imbalance)...")
xgb_model.fit(X_train, y_train, sample_weight=np.array([class_weight_dict[i] for i in y_train]))
print(" XGBoost Training Completed!")

#  Define Base Models (RF + XGB + Logistic Regression)
base_models = [
    ('rf', RandomForestClassifier(
        n_estimators=136,
        max_depth=12,
        min_samples_split=7,
        min_samples_leaf=17,
        max_features='sqrt',
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )),
    ('xgb', xgb_model),
    ('logreg', LogisticRegression(
        solver="lbfgs",
        max_iter=506,
        C=11344.3650513728,
        class_weight="balanced",
        multi_class="multinomial",
        n_jobs=-1
    ))
]

#  Define Meta-Model (LightGBM)
meta_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    learning_rate=0.02893527514545242,
    num_leaves=299,
    max_depth=11,
    min_data_in_leaf=15,
    max_bin=393,
    feature_fraction=0.7447012597505922,
    random_state=42,
    is_unbalance=True,
    min_gain_to_split=0.01,
    device="gpu" if gpu_available else "cpu"
)

#  Define Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

#  Train Stacking Model
print("\n Training Optimized Stacking Classifier...")
start_time = time.time()
stacking_model.fit(X_train, y_train)
end_time = time.time()
print(f" Optimized Stacking Classifier Training Completed in {end_time - start_time:.2f} seconds!")

#  Predictions
print("\n Making predictions...")
y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)

#  Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n {model_name} Results:")
    print(results)
    print(f"\n Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

#  Evaluate Stacked Model
print("\n Evaluating Optimized Stacking Classifier...")
evaluate_model(y_test, y_pred, y_pred_proba, "Stacking Classifier")

print("\n Training & Evaluation Complete! ")


In [ ]:
# Optuna + ExtraaTrees + all columns
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

#  Check GPU Availability
gpu_available = xgb.XGBClassifier().get_params().get('device', 'cpu') == 'cuda'
print(f" GPU Available: {gpu_available}")

#  Load dataset
print("\n Loading dataset...")
df = pd.read_csv("df.csv")
print(f" Dataset Loaded! Shape: {df.shape}")

#  Encode categorical labels
print("\n Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print(" Encoding complete!")

#  Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values
n_classes = len(np.unique(y))  # Number of unique classes

#  Compute Class Weights for Imbalance Handling
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in np.unique(y)}

#  Split dataset
print("\n Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f" Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

#  Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Define XGBoost Model (Updated Parameters)
xgb_model = xgb.XGBClassifier(
    learning_rate=0.0997, max_depth=11, min_child_weight=4, subsample=0.8696,
    colsample_bytree=0.6797, n_estimators=126, 
    tree_method="gpu_hist" if gpu_available else "hist",
    objective="multi:softprob", num_class=n_classes,
    device="cuda" if gpu_available else "cpu", n_jobs=-1
)

#  Train XGBoost with Sample Weights
print("\n Training XGBoost (Handling Class Imbalance)...")
xgb_model.fit(X_train, y_train, sample_weight=np.array([class_weight_dict[i] for i in y_train]))
print(" XGBoost Training Completed!")

#  Define Base Models (RF + XGB + Extra Trees)
base_models = [
    ('rf', RandomForestClassifier(
        n_estimators=184, max_depth=11, min_samples_split=18, min_samples_leaf=4, max_features='sqrt',
        class_weight="balanced", random_state=42, n_jobs=-1
    )),
    ('xgb', xgb_model),
    ('et', ExtraTreesClassifier(
        n_estimators=121, max_depth=12, min_samples_split=3, min_samples_leaf=9, max_features=None,
        class_weight="balanced", random_state=42, n_jobs=-1
    ))
]


#  Define Meta-Model (Updated LightGBM Parameters)
meta_model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.0987, num_leaves=190, max_depth=11, min_data_in_leaf=12,
    max_bin=356, feature_fraction=0.546, random_state=42,
    is_unbalance=True,  # Helps with class imbalance
    min_gain_to_split=0.01,  # Prevents over-pruning
    device="gpu" if gpu_available else "cpu"
)

#  Define Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

#  Train Stacking Model
print("\n Training Optimized Stacking Classifier...")
start_time = time.time()
stacking_model.fit(X_train, y_train)
end_time = time.time()
print(f" Optimized Stacking Classifier Training Completed in {end_time - start_time:.2f} seconds!")

#  Predictions
print("\n Making predictions...")
y_pred = stacking_model.predict(X_test)
y_pred_proba = stacking_model.predict_proba(X_test)

#  Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 {model_name} Results:")
    print(results)
    print(f"\n Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

#  Evaluate Stacked Model
print("\n Evaluating Optimized Stacking Classifier...")
evaluate_model(y_test, y_pred, y_pred_proba, "Stacking Classifier")

print("\n Training & Evaluation Complete! ")
